In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import plotly.express as px
import itertools

In [2]:
df= pd.read_csv('./titanic.csv')

In [3]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [3]:
df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- 'PassengerId' --> id passeggero
- 'Survived' --> sopravvisuti 1, morti 0
- 'Pclass' --> classe del passeggero 1°, 2°, 3°
- 'Name' --> nome del passeggero (non credo sia informativa)
- 'Sex' --> sesso
- 'Age' --> eta
- 'SibSp' --> fratelli o sorelle/compagno a bordo, in generale se aveva qualcuno a bordo
- 'Parch' --> genitori/figli a bordo
- 'Ticket' --> numero del biglietto(credo sia ininfluente come feature visto che si potrebbe utilizzare la classe Pclass oppure Cabin dove credo sia indicato il piano e la cabina)
- 'Fare' --> tariffa, prezzo del biglietto
- 'Cabin' --> cabina
- 'Embarked' --> numero del cadavere, se non è sopravvissuto ed il corpo è stato recuperato (non credo sia informativa)

DOMANDE CHE MI PONGO A CUI CERCHERò DI RISPONDERE CON UN GRAFICO

- è sopravvissuto o no in base alla classe?
- è sopravvissuto o no in base al sesso?
- è sopravvissuto in base all'eta?
- è sopravvissuto se aveva genitori/figli a bordo(SibSp)? penso abbia senso, potremmo dedurre che chi aveva legami a bordo ha cercato di aiutarli mettendo a rischio la propria vita
- è sopravvissuto se aveva genitori/figli a bordo(Parch)?
- è sopravvissuto in base al numero del biglietto?
- è sopravvissuto in base alla cabina?

In [6]:
df.shape

(891, 12)

VALORI NULLI

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

DATASET SBILANCIATO? --> si, 62% verso la classe 0(quindi verso la classe dei morti)

- O classe > --> quindi classe NEGATIVA(TN)
- 1 classe < --> quindi classe POSITIVA(TP)

- TN --> il passeggero muore ed il modello predice che muore
- TP --> il passeggero vive ed il modello predice che vive
- FN --> il passeggero vive ed il modello predice che muore
- FP --> il passeggero muore ed il modello predice che vive

- immaginiamo che questo modello venga richiesto da una nave di salvataggio alla quale viene dato l'elenco di persone della nave e dall'elenco ne mancano il 30%.
- se il modello per il 30% mancante predice:
    - FP(quindi predice che sono vivi ma in realtà sono morti) la barca andrà in giro per il tratto di mare senza trovare nessuno e quindi ad es. il costo potrebbe essere spreco di risorse(equipaggio, carburante, ecc)
    - FN(quindi predice che sono morti ma in realtà non lo sono) la barca non uscirà alla ricerca, risparmierà le risorse, ma condannerà quelle persone a morte certa, quindi in questo caso il "COSTO è >", quindi RECALL

- se invece ci fossero diverse barche affondate in diverse zone bisognerebbe ottimizzare entrambi:
    - i FP poichè ottimizzandoli la barca potra tralasciare una zona e cercare in altre zone e quindi salvare + persone
    - ma anche ottimizzare i FN altrimenti alcune zone verrebbero tralasciate dando le persone per morte, quindi F1(oppure curva ROC per ottimizzare ambo le classi)

In [3]:
# mi creo una nuova colonna con tutti valori 1 per utilizzarla come una sorta di indice
df.insert(0, 'id', 1)

vado a vedere la % di sopravvissuti

In [14]:
df.head()

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
px.pie(df, values='id', names='Survived')

adesso ogni features la confronto con la variabile dipendente y Survived

1° confronto --> Pclass - Survived

# Pclass_Survived

- è sopravvissuto o no(Survived) in base alla classe(Pclass)?
    - Survived --> categorica ma di tipo intera
    - Pclasse --> categorica ma di tipo intera
    - cosa devo fare un bar o un istogramma????

In [9]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
df.head()

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# conteggio dei valori per ogni classe, compresi valori nan
df.Pclass.value_counts(dropna=False)

# da qui possiamo dedurre che + della meta dei campioni risulta in 3° classe

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [10]:
# conteggio in %, sul metodo value_counts() basta settare il parametro normalize=True
df.Pclass.value_counts(dropna=False, normalize=True)

3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64

### vediamo il numero di sopravvissuti in relazione alla classe

In [4]:
# raggruppiamo per Pclass e Survived
pcl_surv_count= df.groupby(['Pclass', 'Survived']).count()['id']

In [5]:
pcl_surv_count

# dal raggruppamento sotto possiamo notare che:
# in 1° classe ci sono + sopravvissuti
# in 2° classe c'è un ex aequo(pari valori)
# in 3° classe ci sono + morti

Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
Name: id, dtype: int64

sotto, tutto riportato in %

In [8]:
pcl_surv_count_perc = pcl_surv_count.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

In [9]:
pcl_surv_count_perc

Pclass  Survived
1       0           37.037037
        1           62.962963
2       0           52.717391
        1           47.282609
3       0           75.763747
        1           24.236253
Name: id, dtype: float64

fin qui potremmo dedurre che:

- la maggior parte dei passeggeri era in 3° classe
- la 2° classe è anche la classe con la % di morte > mentre nella 1° classe la % è <

-------------------------------------------

sotto cerchiamo di graficare tutto tramite un grafico(anche se i valori già li abbiamo trovati)

In [ ]:
# grafichiamo "pcl_surv_count_perc" cioè classe e sopravvisuti in %

# pcl_surv_count_perc2 = pd.DataFrame(pcl_surv_count.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))).reset_index()
# pcl_surv_count_perc2
# pcl_surv_count_perc2.columns= ['Pclass','Survived','Perc %']
# pcl_surv_count_perc2['Pclass']= pcl_surv_count_perc2['Pclass'].apply(str)
# pcl_surv_count_perc2['Survived']= pcl_surv_count_perc2['Survived'].apply(str)
# px.bar(pcl_surv_count_perc2, x='Pclass', y='Perc %', color='Survived',text_auto=True)

--------------------------------------------

In [13]:
# trasformo la groupby di sopra in dataframe e resetto l'indice
df_pcl_surv= pd.DataFrame(df.groupby(['Pclass', 'Survived']).count()['id']).reset_index()
df_pcl_surv

,Pclass,Survived,id
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


In [14]:
# rinominiamo le colonne di df_pcl_surv, soprattutto per id che la portiamo a count
df_pcl_surv.columns= ['Pclass', 'Survived', 'Counts']
df_pcl_surv

# cosa posso dire?
# - in classe 2 i valori mi sembrano invariati --> sono quasi gli stessi
# - in classe 1 su un totale di 216 passeggeri:
#   - 136 si salvano, quindi + del 62%(circa)
#   - 80 muoiono, circa il 37%(circa)
# - in classe 3 su un totale di 491 passeggeri:
#   - 119 si salvano, quindi il 24%(circa)
#   - 372 muoiono, quindi il 76%(circa)

# potremmo quindi dire che in 1° classe c'è' > probabilità di salvarsi(la stessa cosa la deduco facendo il plot sotto) --> ok, però il numero di soggetti della 3° classe è anche > rispetto ai soggetti delle altre classi, questo potrebbe influenzare? Non credo poichè la % dei sopravvisuti o non la stiamo calcolando solo sui soggetti della 3° classe, quindi classe x classe, non sul totale dei soggetti

,Pclass,Survived,Counts
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


In [15]:
# ??????? poi qui convertiamo le colonne Pclass e Survived in str, xkè?? forse perchè utilizzo un bar?? allora avrei potuto utilizzare un istogramma senza convertire le features, o sbaglio?? ??????????

In [16]:
df_pcl_surv['Pclass']= df_pcl_surv['Pclass'].apply(str)
df_pcl_surv['Survived']= df_pcl_surv['Survived'].apply(str)

In [17]:
px.bar(df_pcl_surv, x='Pclass', y='Counts', color='Survived',text_auto=True)

COSA NOTIAMO??

- la classe 1 e 2 hanno meno occorrenze, probabilmente poichè il prezzo del biglietto era + alto
    - la % di sopravvissuti e non sopravvissuti è bilanciata sia per la classe 1 che per la classe 2, forse in 1° classe ci sono + occorrenze verso i sopravvissuti
- la classe 3 è quella che ha + occorrenze di tutti, probabilmente per il prezzo del biglietto + basso
    - notiamo poi che qui la % di morti è molto + alta dei sopravvissuti, + del triplo o giù di lì, quindi potremmo dedurre che probabilmente chi aveva la 3° aveva + probabilità di morire, forse perchè le cabine erano situate + verso il basso e quindi le prime ad essere allagate durante lo sprofondamento


    ?????? bisogna anche controllare il prezzo dei biglietti per le varie classi e capire se in 3° classe costano meno o più che nelle altre classi, in questo modo capiamo il motivo per cui ci sono stati + biglietti venduti in 3° classe ????? --> forse potrei raggruppare i prezzi dei biglietti per classe e per ogni classe poi fare la media dei prezzi e vedere se la media del prezzo del biglietto di ogni singola classe differisce l'una dall'altro, l'ho fatto dove divido il prezzo dei biglietti di gruppo

In [18]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

# Sex_Survived

- è sopravvissuto o no(Survived) in base al sesso(Sex)?
    - Survived --> categorica ma di tipo intera
    - Sex --> categorica ma di tipo object
    - cosa devo fare un bar o un istogramma????

In [20]:
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


------------------------------------------
sotto ragioniamo in %

In [23]:
# - level=0 penso sia una sorta di reset_index() specifica il 1° indice, quindi la 1° colonna(almeno credo)
# poi gli passo 'id':'count' sotto forma di dizionario in modo da restituirmi solo i valori di quella colonna, altrimenti avrebbe restituito tutto il dataset
df_sex_surv_perc= df.groupby(['Sex', 'Survived']).agg({'id':'count'}).groupby(level=0).apply(lambda x: 100*x/x.sum())

# passandogli level=0 restituisce le % sulla colonna 0, cioè Sex --> quindi mi restituisce la % di donne, sopravvissute e non e la % di uomini, sopravvissuti e non
# passandogli level=1 restituisce le % sulla colonna 1, cioè Survived --> quindi mi restituisce la % di tutti i morti(sia uomo che donna) e poi dei vivi(sia uomo che donne) --> quindi da questo otterremo che:
    # - donne
    #   - 25% morte
      # - 74% vive
    # - uomini
    #   - 80% morti
    #   - 18% vivi


# possiamo quindi dedurre che muoino + uomini e meno donne, forse perchè le donne insime ai bambini hanno precedenza 

In [24]:
df_sex_surv_perc

id
Sex    Survived           
female 0         25.796178
       1         74.203822
male   0         81.109185
       1         18.890815

------------------------------------------

In [26]:
# conteggio dei valori compresi valori nan
df.Sex.value_counts(dropna=False, normalize=False)

male      577
female    314
Name: Sex, dtype: int64

In [31]:
# conteggio dei valori compresi valori nan, in %
df.Sex.value_counts(dropna=False, normalize=True)

male      0.647587
female    0.352413
Name: Sex, dtype: float64

In [20]:
df_sex_surv= pd.DataFrame(df.groupby(['Sex', 'Survived']).count())['id'].reset_index()

In [21]:
df_sex_surv

,Sex,Survived,id
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


In [22]:
# visto che Survived è categorica di tipo intero e la stiamo confrontando con Sex che è categorica di tipo object portiamo Survived a str o object
df_sex_surv['Survived']= df_sex_surv['Survived'].apply(str)

In [23]:
# rinominiamo le colonne
df_sex_surv.columns=['Sex','Survived','Counts']

In [24]:
px.bar(df_sex_surv, x='Sex', y='Counts', color='Survived', text_auto=True)

COSA NOTIAMO??

- abbiamo una prevalenza di occorenze per quanto riguarda il sesso male, quindi molti + uomini a bordo, quasi il doppio
    - dalla parte sex female, cioè donne, notiamo una % maggiore di sopravvissuti, forse perchè gli uomini erano impegnati a salvare altre persone, forse per una questione di sesso(precedenza per le donne), forse perchè le donne avevano figli quindi dando prencedenza ai bambini automaticamente si dà precedenza anche alle madri(donne)
    - mentre dalla parte sex male, cioè uomini, la percentuali di morti è molto alta, quasi 5 volte tanto, credo per gli stessi motivi di female ma all'opposto

N.B. --> si potrebbe comparare Sex, Pclass e Surived per capire se gli uomini sono morti in maggioranza poichè erano in 3° classe dove i morti sono stati maggiori.....lo faccio sotto ed abbiamo che su 347 uomini in 3° classe 300 sono morti --> si potrebbe dedurre che la 3° classe sia stata un causa di morte, soprattutto per gli uomini, per le donne un po' meno forse perchè a loro si è dato precedenza(insieme ai bambini)

--------------------

In [9]:
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


---------------------------

In [ ]:
# .apply(lambda x: 100*x/x.sum())

In [10]:
df_sex_pclass= df.groupby(['Sex','Pclass'])['id'].count()

In [11]:
df_sex_pclass.groupby(level=1).apply(lambda x: 100*x/x.sum())

# qui sotto abbiamo fatto un check sul sesso dei passeggeri in relazione alla classe e notiamo che la differenza maggiore la si nota in 3° classe dove il numero di uomini è del 70% contro il 30% di donne, per quanto riguarda le altre classi possiamo affermare che la differenza tra uomini e donne è abbastanza bilanciata(circa 40% donne e circa 60% uomini)

Sex     Pclass
female  1         43.518519
        2         41.304348
        3         29.327902
male    1         56.481481
        2         58.695652
        3         70.672098
Name: id, dtype: float64

----------------------------

sotto penso sia la stessa cosa, ma con l'aggiunta della feature Survived e con l'aggiunta della colonna %

In [72]:
# df_sex_pclass_surv_perc= df.groupby(['Sex','Pclass','Survived']).agg({'id':'count'})
df_sex_pclass_surv_perc= df.groupby(['Sex','Pclass','Survived'])['id'].count()

In [88]:
df_sex_pclass_surv_perc.groupby(level=1).apply(lambda x: 100*x/x.sum())

# N.B. level è molto importante, in pratica indica la colonna da cui partire per trovare le corrispondenze, ad es.
# - level=0 la ricerca parte dalla colonna Sex quindi raggruppa per le classi female e male, che sommate danno il 100%
# - level=1 la ricerca parte dalla colona Pclass quindi raggruppa per le classi 1,2 e 3, che sommate(singolarmente, cioè 1 di female ed 1 di male, 2 di female e 2 di male, e così via) danno il 100%
# - level=2 la ricerca parte dalla colonna Survived, quindi raggruppa le classi 0 ed 1, che sommate danno il 100%

Sex     Pclass  Survived
female  1       0            1.388889
                1           42.129630
        2       0            3.260870
                1           38.043478
        3       0           14.663951
                1           14.663951
male    1       0           35.648148
                1           20.833333
        2       0           49.456522
                1            9.239130
        3       0           61.099796
                1            9.572301
Name: id, dtype: float64

--------------------

N.B. --> cmq in generale mi sembra che:

- per le variabili discrete si utilizzino gli istogrammi --> quindi i valori discreti vengono spalmati sull'asse x e poi sull'asse y viene fatto il conteggio delle occorrenze, con la legenda categorica di lato
    - Pclass(valori discreti) che è la variabile indipendente del nostro dataset che vogliamo confrontare con la variabile dipendente(Survived, che sarà la legenda) e poi ci prendiamo un'altra variabile(se non ce l'abbiamo la creiamo, come abbiamo fatto ora) per contare le occorrenze di entrambe le variabili(Pclass e Survived)

- per le variabili categoriche si utilizzino i barplot, a prescindere dal tipo tanto poi lo si può convertire il tipo --> sull'asse x mettiamo le categorie e sull'asse y il conteggio, con la legenda categorica di lato
    - Sex(valori categorici) che è la variabile indipendente del nostro dataset che vogliamo con la variabile dipendente(Survived, che sarà la legenda) e poi ci prendiamo un'altra variabile(se non ce l'abbiamo la creiamo, come abbiamo fatto ora) per contare le occorrenze di entrambe le variabili(Sex e Survived)

DA VEDERE
- ????????? fare un'istogramma sull'eta, distribuzione Survived e NoSurvived --> farlo insieme ??????
- in contrapposizione alla colonna id che mi sono creato posso creare un pie chart fare una groupby sulla variabile che mi interessa, in questo caso Survived, e poi fare una count su Survived --> quindi lavoro su un'unica features contanfo le ooccorenze di Survived
- VARIABILI DISCRIMINANTI --> 3° CLASSE e SESSO MASCHIO(fin ora)

Age_Survived
- è sopravvissuto in base all'eta?

In [25]:
df.head()

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [27]:
px.histogram(df, x='Age', color='Survived')

DALL'ISTOGRAMMA SOPRA ABBIAMO UNA DOPPIA CONFERMA SUL FATTO CHE LE PERSONE ANZIANE HANNO MENO PROBABILITà DI SALVARSI RISPETTO AI RAGAZZINI, INFATTI DA (CIRCA)64 ANNI IN POI NON VEDIAMO NESSUNA BARRA ROSSA CHE STA AD INDICARE I SOPRAVVISSUTI MENTRE VEDIAMO SOLO BARRE BLU CHE STANNO AD INDICARE I MORTI(NON SOPRAVVISSUTI)

LINK CABINA --> http://www.titanicdiclaudiobossi.com/Html/Cabine%20del%20Titanic_158.htm

CAMPIONI Age <= 16

In [28]:
df_age_min_16= df[df['Age']<=16]

In [29]:
df_age_min_16.groupby(['Survived'])['id'].count()

Survived
0    45
1    55
Name: id, dtype: int64

In [30]:
df_age_min_16= pd.DataFrame(df_age_min_16.groupby(['Survived'])['id'].count()).reset_index()
df_age_min_16

,Survived,id
0,0,45
1,1,55


In [31]:
df_age_min_16.columns= ['Survived','Counts']

In [32]:
df_age_min_16

,Survived,Counts
0,0,45
1,1,55


In [33]:
px.pie(df_age_min_16, values='Counts', names='Survived')

cosa possiamo dedurre?

- il piechart Age <= 16 non mi sembra molto informatico visto che gli esseri umani di eta compresa tra 0 a 16 hanno (quasi) le stesse probabilità di salvarsi o di morire rispetto alla distribuzione totale di Survived
- vi è una piccola maggiorazione verso i sopravvissuti --> forse perchè i giovani hanno la precedenza sugli adulti(si dice sempre prima donne e bambini) o forse perchè i genitori + abbienti hanno portato i figli con se e poichè erano in 1° classe hanno avuto + probabilità di salvarsi???

CAMPIONI Age > 50

In [34]:
df_age_max_50= df[df['Age']>=50]

In [35]:
df_age_max_50= pd.DataFrame(df_age_max_50.groupby(['Survived']).count()['id'].reset_index())

In [36]:
df_age_max_50.columns= ['Survived', 'Counts']

In [37]:
px.pie(df_age_max_50, values='Counts', names='Survived')

cosa possiamo dedurre?

- il piechart Age >= 50 non mi sembra molto informatico visto che la distribuzione è molto simile a quella totale di Survived

CAMPIONI Age > 16 e < 50

In [38]:
df_age_max_16_min_50= df[(df['Age']>16) & (df['Age']<50)]

In [39]:
df_age_max_16_min_50

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,1,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,1,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,1,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [40]:
df_age_max_16_min_50= pd.DataFrame(df_age_max_16_min_50.groupby(['Survived']).count()['id'].reset_index())

In [41]:
df_age_max_16_min_50.columns= ['Survived', 'Counts']

In [42]:
px.pie(df_age_max_16_min_50, values='Counts', names='Survived')

cosa possiamo dedurre?

- il piechart Age > 16 e < 50 non mi sembra molto informatico visto che la distribuzione è molto simile a quella totale di Survived

RICAPITOLANDO possiamo dire che forse c'è un leggero incremento di sopravvissuti nel range < 16, forse perchè i bambini hanno la precedenza sugli adulti 

### RECAP


Survived
38.4 --> sopravvissuti
61.6 --> deceduti

-------------------------

Age<=16
45 --> sopravvissuti
55 --> deceduti

Age>16 & Age<50
38.5 --> sopravvissuti
61.5 --> deceduti

Age>=50
36.5 --> sopravvissuti
63.5 --> deceduti

Sibsp_Survived

- è sopravvissuto se aveva qualcuno a bordo(SibSp)? penso abbia senso, potremmo dedurre che chi aveva legami a bordo ha cercato di aiutarli mettendo a rischio la propria vita
- dobbiamo conforntarla con Survived anch'essa di tipo categorica e di tipo intera

NOTE --> secondo me è una variabile categorica anche se di tipo int64, mi muoverò verso tal senso trattandola come categorica
- quindi proiettereremo sull'asse delle x le variabili categoriche di SibSp
- sull'asse delle y il conteggio dei valori
- e come legenda utilizzeremo sempre la feature Survived

In [43]:
df.SibSp.value_counts(dropna=False)

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [44]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [45]:
df.head()

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [46]:
df.groupby(['SibSp','Survived']).count()['id']

SibSp  Survived
0      0           398
       1           210
1      0            97
       1           112
2      0            15
       1            13
3      0            12
       1             4
4      0            15
       1             3
5      0             5
8      0             7
Name: id, dtype: int64

In [47]:
df_sibsp_surv= pd.DataFrame(df.groupby(['SibSp','Survived']).count()['id']).reset_index()

In [48]:
df_sibsp_surv

,SibSp,Survived,id
0,0,0,398
1,0,1,210
2,1,0,97
3,1,1,112
4,2,0,15
5,2,1,13
6,3,0,12
7,3,1,4
8,4,0,15
9,4,1,3


In [49]:
df_sibsp_surv.columns= ['SibSp', 'Survived', 'Counts']

In [50]:
df_sibsp_surv

,SibSp,Survived,Counts
0,0,0,398
1,0,1,210
2,1,0,97
3,1,1,112
4,2,0,15
5,2,1,13
6,3,0,12
7,3,1,4
8,4,0,15
9,4,1,3


In [51]:
# questo step posso anche ometterlo, però omettendolo come legenda otterremo valori in range da 0 ad 1 e non come singole classi 0 o 1 --> ????? perchè le porto ad str ???????
df_sibsp_surv['SibSp']= df_sibsp_surv['SibSp'].apply(str)
df_sibsp_surv['Survived']= df_sibsp_surv['Survived'].apply(str)

In [52]:
px.bar(df_sibsp_surv, x='SibSp', y='Counts', color='Survived',text_auto=True)

SibSp penso sia poco informativa per i seguenti motivi

- come 1° cosa potremmo dire che quasi tutti(il 90% circa) erano soli o al max avevano un parente/figlio e forse non c'è relazione tra la sopravvivenza e la morte visto che le proporzioni sono quasi uguali
    - nessun famigliare --> 210 sopravvissuti, 398 morti
    - 1 famigliare --> 112 sopravvissuti, 97 morti
- (anche se i campioni sono pochi) potremmo dedurre che al crescere dei famigliari a bordo la probabilità di sopravvivere si abbassa di molto, quasi inesistene, questo perchè
    - o si cerca di salvarli tutti e quindi si muore tutti visto che il numero è elevato
    - oppure chi viaggia con diversi famigliari sono in 3° classe cioè il posto + in basso della nave e quindi + soggetto all'affondamento

Forse la feature + rilevante è data da Pclass che indica la classe dello scafo

Parch_Survived

In [53]:
df.columns

Index(['id', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [54]:
df.groupby(['Parch', 'Survived']).count()['id']

Parch  Survived
0      0           445
       1           233
1      0            53
       1            65
2      0            40
       1            40
3      0             2
       1             3
4      0             4
5      0             4
       1             1
6      0             1
Name: id, dtype: int64

In [55]:
df_parch_surv= pd.DataFrame(df.groupby(['Parch', 'Survived']).count()['id']).reset_index()

In [56]:
df_parch_surv.columns= ['Parch', 'Survived', 'Counts']

In [57]:
# converto in stringa, stessa cosa di SibSp --> capire perchè
df_parch_surv['Parch']= df_parch_surv['Parch'].apply(str)
df_parch_surv['Survived']= df_parch_surv['Survived'].apply(str)

In [109]:
px.bar(df_parch_surv, x='Parch', y='Counts', color='Survived', text_auto=True)

cosa comprendiamo?

in generale possiamo notare che la maggior parte di chi si era imbarcato lo ha fatto da solo, questo lo notiamo da
- SibSp --> fratelli o sorelle a bordo / compagno a bordo(max 0 o 1)
- Parch --> genitori/figli a bordo(max 0,1 e 2)

VOGLIO CONFRONTARE Pclass e Fare, cioè se la classe è relazionata al prezzo del biglietto

In [59]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [60]:
df.groupby(['Fare', 'Pclass']).count()['id']

Fare      Pclass
0.0000    1         5
          2         6
          3         4
4.0125    3         1
5.0000    1         1
                   ..
227.5250  1         4
247.5208  1         2
262.3750  1         2
263.0000  1         4
512.3292  1         3
Name: id, Length: 255, dtype: int64

In [61]:
ciao= pd.DataFrame(df.groupby(['Fare', 'Pclass']).count()['id']).reset_index()

In [62]:
ciao

,Fare,Pclass,id
0,0.0000,1,5
1,0.0000,2,6
2,0.0000,3,4
3,4.0125,3,1
4,5.0000,1,1
...,...,...,...
250,227.5250,1,4
251,247.5208,1,2
252,262.3750,1,2
253,263.0000,1,4


In [63]:
ciao.columns= ['Fare', 'Pclass', 'Counts']

In [64]:
px.bar(ciao, x='Pclass', y='Counts', color='Fare', text_auto=True)
# non mi sembra che la classe sia relazionata al costo dei biglietti visto che anche se i valori dei prezzi + alti sono nella classe 1 ci sono anche tantissimi valori bassi(quindi prezzi bassi) nella classe 1, addirittura ci sono 5 soggeti con prezzo 0

In [65]:
df.Fare.value_counts(dropna=True)

8.0500     43
13.0000    42
7.8958     38
7.7500     34
26.0000    31
           ..
35.0000     1
28.5000     1
6.2375      1
14.0000     1
10.5167     1
Name: Fare, Length: 248, dtype: int64

In [66]:
df.dtypes

id               int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

CERCHIAMO DI LAVORARE SUI BIGLIETTI(Ticket) --> ci sono biglietti di gruppo quindi lo stesso numero di Ticket è assegnato a + soggetti ed in questo caso il prezzo per ogni soggetto verrà applicato il prezzo complessivo del ticket e non il prezzo del singolo ticket --> quindi quello che bisognerà fare è individuare i ticket di gruppo e fare il conteggio delle persone assegnate a quel ticket di gruppo e poi dividere il prezzo del ticket per il numero di soggetti applicati a quello specifico ticket

-----------
MOSTRIAMO I BIGLIETTI CHE RAGGRUPPANO IL > NUMERO DI PERSONE, DA QUI POI MOSTRIAMO LA CLASSE DEL BIGLIETTO E SE SONO SOPRAVVISSUTI OPPURE NO

In [30]:
df.groupby(['Pclass','Ticket','Survived'])['id'].count().sort_values(ascending=False)
# dalla group by possiamo notare che:
    # - il numero >(da 7 a 4 persone per biglietto) di biglietti di gruppo è in 3° classe e la > parte di essi muore

Pclass  Ticket              Survived
3       347082              0           7
        CA. 2343            0           7
        3101295             0           6
        347088              0           6
        CA 2144             0           6
        382652              0           5
        1601                1           5
2       S.O.C. 14879        0           5
3       349909              0           4
        W./C. 6608          0           4
        2666                1           4
1       113760              1           4
3       4133                0           4
1       PC 17572            1           3
        PC 17755            1           3
        110152              1           3
3       347742              1           3
        345773              0           3
        LINE                0           3
1       24160               1           3
3       347077              1           3
1       13502               1           3
        PC 17757            1          

-----------

In [15]:
df.groupby(['Sex','Survived'])['id'].count().groupby(level=0).apply(lambda x: 100*x/x.sum())
# ho aggiunto le %, qualora le volessi togliere basta eliminare il codice dalla 2° groupby in poi
# notiamo che (circa):
# - il 74% delle donne si salva
# - solo il 18% degli uomini si salva
# potremmo dedurre che visto che circa l'81% degli uomini è morto e circa il 90% degli uomini che era in 3° classe è morto le discriminanti potrebbero essere:
#   - classe = 3
#   - sesso = uomo

Sex     Survived
female  0           25.796178
        1           74.203822
male    0           81.109185
        1           18.890815
Name: id, dtype: float64

In [31]:
df.groupby(['Pclass','Sex','Survived'])['id'].count().sort_values(ascending=False)
# da qui notiamo che:
    # - in 3° classe
    #     - su un totale di 347 uomini
    #         - 300 sono morti
    #         - 47 sono vivi
    #     - su un totale di 144 donne
    #         - 72 sono morti
    #         - 72 sono vivi
    # - in 2° classe
    #     - su un totale di 108 uomini
    #         - 91 sono morti
    #         - 17 sono vivi
    #     - su un totale di 76 donne
    #         - 6 sono morti
    #         - 70 sono vivi
    # - in 1° classe
    #     - su un totale di 122 uomini
    #         - 77 sono morti
    #         - 45 sono vivi
    #     - su un totale di 94 donne
    #         - 3 sono morti
    #         - 91 sono vivi

Pclass  Sex     Survived
3       male    0           300
1       female  1            91
2       male    0            91
1       male    0            77
3       female  0            72
                1            72
2       female  1            70
3       male    1            47
1       male    1            45
2       male    1            17
        female  0             6
1       female  0             3
Name: id, dtype: int64

In [ ]:
df.loc[(df['Sex']=='female') & (df['Pclass']==3), ['Sex','Pclass','Survived']]

In [44]:
df.groupby(['Sex','Pclass','Survived'])['id'].count().sort_values(ascending=False)

Sex     Pclass  Survived
male    3       0           300
female  1       1            91
male    2       0            91
        1       0            77
female  3       0            72
                1            72
        2       1            70
male    3       1            47
        1       1            45
        2       1            17
female  2       0             6
        1       0             3
Name: id, dtype: int64

prima di fare la groupby sotto, contenente anche la feature Age, dovrei cercare di raggruppare la feature Age, visto che si tratta di una variabile discreta e quindi avendo un'infinità di valori ci conviene fare dei range --> penso che cmq sia stata fatta con un plot sopra --> vediamo adesso se riusciamo a farla con dei filtri tramite codice*

io direi di creare delle variabili categoriche tipo 

In [45]:
px.histogram(df, x='Age')

In [48]:
px.histogram(df, x='Age', nbins=80)

In [40]:
df.groupby(['Age','Pclass','Survived'])['id'].count().sort_values(ascending=False)

Age    Pclass  Survived
22.00  3       0           15
21.00  3       0           15
28.00  3       0           12
20.00  3       0           12
18.00  3       0           12
19.00  3       0           11
25.00  3       0           11
26.00  3       0           10
16.00  3       0            9
24.00  3       0            9
35.00  1       1            9
30.00  3       0            9
32.00  3       0            7
17.00  3       0            7
36.00  1       1            7
29.00  3       0            7
33.00  3       0            7
23.00  2       0            6
27.00  3       1            6
24.00  2       1            6
2.00   3       0            6
9.00   3       0            6
35.00  3       0            5
25.00  2       0            5
36.00  3       0            5
28.00  2       1            5
39.00  3       0            5
24.00  1       1            5
40.00  3       0            5
48.00  1       1            5
34.00  2       0            5
               1            5
30.00  1       1

In [49]:
df.head()

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
df.isna().sum()

id               0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [52]:
# eliminiamo la colonna Cabin, troppi nan values, anche rimpiazzandoli falseremmo il dataset visto che sono molti --> N.B. non serve nemmeno eliminarla, basta escluderla in fase di cross validation
df.drop('Cabin', axis=1, inplace=True)

In [55]:
df.head(1)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,S


# CROSS VALDIATION

In [4]:
# library
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from statistics import mode, mean, median
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

- prima della cross validation devo sistemare la feature Fare(prezzo) in relazione a Ticket(sopra abbiamo scritto un'appunto), passarla così non ha senso e non andrebbe bene al modello. Poiche essendoci dei biglietti di gruppo(quindi un di biglietto assegnato a + persone) bisogna dividere il prezzo(Fare) totale di quel biglietto per il numero di passeggeri --> faccio un value_counts() su Ticket e vedo i biglietti corrispendenti a + di un singolo utente

- in questo modo avrei capito se il prezzo del ticket era totale o per singolo persona, penso sia giusto che per ogni singola persona ci sia il prezzo per singola persona, non il totale --> poi abbiamo notato che da 7 a 4 persone sono in 3° classe, almeno quasi tutti

In [90]:
df.Ticket.value_counts()

347082                7
CA. 2343              7
1601                  7
3101295               6
CA 2144               6
347088                6
S.O.C. 14879          5
382652                5
LINE                  4
PC 17757              4
17421                 4
349909                4
113760                4
4133                  4
113781                4
W./C. 6608            4
2666                  4
19950                 4
347077                4
C.A. 31921            3
PC 17572              3
C.A. 34651            3
363291                3
F.C.C. 13529          3
345773                3
248727                3
24160                 3
29106                 3
SC/Paris 2123         3
35273                 3
371110                3
230080                3
PC 17760              3
239853                3
PC 17582              3
347742                3
110152                3
13502                 3
110413                3
PC 17755              3
PC 17558              2
237736          

In [9]:
# visto che il Ticket numero 347082 si ripete 7 volte deduco che sia un biglietto multiplo,(solo i ticket che si ripetono 1 sola volta sono singoli), provo a visualizzarlo per avere + chiara la situazione
df.loc[df['Ticket']=='347082',['Pclass','Name','Ticket','Fare']]

,Pclass,Name,Ticket,Fare
13,3,"Andersson, Mr. Anders Johan",347082,31.275
119,3,"Andersson, Miss. Ellis Anna Maria",347082,31.275
541,3,"Andersson, Miss. Ingeborg Constanzia",347082,31.275
542,3,"Andersson, Miss. Sigrid Elisabeth",347082,31.275
610,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",347082,31.275
813,3,"Andersson, Miss. Ebba Iris Alfrida",347082,31.275
850,3,"Andersson, Master. Sigvard Harald Elias",347082,31.275


------------------------------------------------

In [10]:
# stessa cosa per il Ticket 349909
df.loc[df['Ticket']=='349909',['Pclass','Name','Ticket','Fare']]

,Pclass,Name,Ticket,Fare
7,3,"Palsson, Master. Gosta Leonard",349909,21.075
24,3,"Palsson, Miss. Torborg Danira",349909,21.075
374,3,"Palsson, Miss. Stina Viola",349909,21.075
567,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",349909,21.075


CONVERSIONE DEI VALORI DELLA FEATURE Sex --> lo faccio visto che in generale gli algoritmi comprendono meglio i numeri rispetto ai caratteri(non è necessario per tutti gli algortmi, ad es. gli alberi)
- male --> 1
- female --> 0

N.B. --> in generale, la conversione di una variabile categorica binaria, da stringa a numerica, lo si può fare fuori dalla cross validation, come abbiamo fatto sotto, ma nel nostro caso specifico visto che vogliamo esercitarci e capire come farlo in una cross validation, lo facciamo all'interno della cross validation --> altrimenti, come abbiamo detto e fatto sotto(per prova, senza eseguire la cella), possiamo farlo al di fuori della cross validation

In [19]:
map_sex= {'male': 1, 'female': 0}
df['Sex_conv']= df['Sex'].map(map_sex)

In [38]:
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_conv
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0


In [55]:
# adesso che sappiamo che ci sono Ticket di gruppo visto che li devo dividere controlliamo la feature Fare(prezzo) in modo da controllare se vi sono anomalie --> risultano delle anomalie, abbiamo 15 valori negativi, come gestirli??, vediamo sotto a cosa corrispondono --> li cancello, in pratica sarebbero i biglietti senza prezzo
df['Fare'].value_counts().sort_index()

0.0000      15
4.0125       1
5.0000       1
6.2375       1
6.4375       1
6.4500       1
6.4958       2
6.7500       2
6.8583       1
6.9500       1
6.9750       2
7.0458       1
7.0500       7
7.0542       2
7.1250       4
7.1417       1
7.2250      12
7.2292      15
7.2500      13
7.3125       1
7.4958       3
7.5208       1
7.5500       4
7.6292       1
7.6500       4
7.7250       1
7.7292       1
7.7333       4
7.7375       2
7.7417       1
7.7500      34
7.7750      16
7.7875       1
7.7958       6
7.8000       1
7.8292       2
7.8542      13
7.8750       1
7.8792       4
7.8875       1
7.8958      38
7.9250      18
8.0292       1
8.0500      43
8.1125       1
8.1375       1
8.1583       1
8.3000       1
8.3625       1
8.4042       1
8.4333       1
8.4583       1
8.5167       1
8.6542       1
8.6625      13
8.6833       1
8.7125       1
8.8500       1
9.0000       2
9.2167       1
9.2250       2
9.3500       2
9.4750       1
9.4833       1
9.5000       9
9.5875       2
9.8250    

In [60]:
# grafico alcune features con Fare = 0, sempre per avere un'idea generale e rendermi conto meglio
df.loc[df['Fare']==0,['Pclass','Name','Ticket','Fare','Age']]

,Pclass,Name,Ticket,Fare,Age
179,3,"Leonard, Mr. Lionel",LINE,0.0,36.0
263,1,"Harrison, Mr. William",112059,0.0,40.0
271,3,"Tornquist, Mr. William Henry",LINE,0.0,25.0
277,2,"Parkes, Mr. Francis ""Frank""",239853,0.0,NaN
302,3,"Johnson, Mr. William Cahoone Jr",LINE,0.0,19.0
413,2,"Cunningham, Mr. Alfred Fleming",239853,0.0,NaN
466,2,"Campbell, Mr. William",239853,0.0,NaN
481,2,"Frost, Mr. Anthony Wood ""Archie""",239854,0.0,NaN
597,3,"Johnson, Mr. Alfred",LINE,0.0,49.0
633,1,"Parr, Mr. William Henry Marsh",112052,0.0,NaN


In [5]:
# dopo aver visto a cosa corrispondono li cancello, eliminare interamente le righe con Fare = 0

# il codice sotto l'avrei anche potuto scrivere cosi senza assegnare la variabile fare_drop_zero, sotto il codice giusto
# df.drop(df[df['Fare']==0].index, inplace=True)

fare_drop_zero = df[df['Fare']==0].index
df.drop(fare_drop_zero, inplace=True)
df.head(5)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


feature FARE --> bisogna trovare tutte le persone che hanno un biglietto unico e dividere il valore di fare di ogni persona per il numero di persone che hanno lo stesso biglietto

In [50]:
df.loc[df['Ticket']=='347082',['Pclass','Name','Ticket','Fare']]

,Pclass,Name,Ticket,Fare
13,3,"Andersson, Mr. Anders Johan",347082,31.275
119,3,"Andersson, Miss. Ellis Anna Maria",347082,31.275
541,3,"Andersson, Miss. Ingeborg Constanzia",347082,31.275
542,3,"Andersson, Miss. Sigrid Elisabeth",347082,31.275
610,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",347082,31.275
813,3,"Andersson, Miss. Ebba Iris Alfrida",347082,31.275
850,3,"Andersson, Master. Sigvard Harald Elias",347082,31.275


In [7]:
df_pcl_surv= pd.DataFrame(df.groupby(['Pclass', 'Survived']).count()['id']).reset_index()
df_pcl_surv

,Pclass,Survived,id
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


In [5]:
# df.groupby(['Pclass','Ticket','Survived','Fare'])['id'].count().sort_values(ascending=False)
# sort_values(ascending=False), codice sotto, avrei potuto anche evitarlo, l'ho lasciato per mostrare che si potesse utilizzare in combinazione, come sopra
df_pcl_tick_fare_count= pd.DataFrame(df.groupby(['Pclass','Ticket','Survived','Fare'])['id'].count().sort_values(ascending=False).reset_index())

In [6]:
df_pcl_tick_fare_count.columns= ['Pclass', 'Ticket', 'Survived', 'Fare', 'Count']

In [9]:
# in questo modo abbiamo ottenuto un dataset contenente il numero di volte che un determinato Ticket si ripete, qualora fosse > di 1(la colonna Count) si tratta di un biglietto globale(contiene + passeggeri)
df_pcl_tick_fare_count.head(10)

,Pclass,Ticket,Survived,Fare,Count
0,3,CA. 2343,0,69.5500,7
1,3,347082,0,31.2750,7
2,3,3101295,0,39.6875,6
3,3,347088,0,27.9000,6
4,3,CA 2144,0,46.9000,6
5,3,1601,1,56.4958,5
6,3,382652,0,29.1250,5
7,2,S.O.C. 14879,0,73.5000,5
8,3,4133,0,25.4667,4
9,3,349909,0,21.0750,4


In [32]:
# adesso seleziono count > di 2 --> l'ho tolta poichè in questo modo non seleziono solo i biglietti globali ma li seleziono tutti, tanto per i biglietti singoli il prezzo resta lo stesso visto che count = 1 e quindi Fare è diviso 1 che restituisce quindi il valore di Fare
# df_pcl_tick_fare_count= df_pcl_tick_fare_count.loc[df_pcl_tick_fare_count['Count']>=2,['Pclass', 'Ticket', 'Survived', 'Fare', 'Count']]

In [ ]:
# quello che vogliamo è dividere i valori della colonna Count con i valori della colonna Fare(naturalemnte in corrispondenza), in questo otterremo il prezzo unitario del singolo biglietto --> anche nel caso di Count = 1 restituirà il prezzo del biglietto unitario poichè restituira il prezzo(Fare) diviso Count che è 1, quindi il prezzo del biglietto singolo

In [7]:
# aggiungo una nuova colonna, Sing_Fare che conterra il prezzo del biglietto(Fare) / il numero di persone(Count), in questo modo ottengo il prezzo unitario e non più quello globale
df_pcl_tick_fare_count['Sing_Fare']= df_pcl_tick_fare_count.apply(lambda row: row.Fare / row.Count, axis=1)

In [9]:
df_pcl_tick_fare_count.head(10)

,Pclass,Ticket,Survived,Fare,Count,Sing_Fare
0,3,347082,0,31.2750,7,4.467857
1,3,CA. 2343,0,69.5500,7,9.935714
2,3,3101295,0,39.6875,6,6.614583
3,3,347088,0,27.9000,6,4.650000
4,3,CA 2144,0,46.9000,6,7.816667
5,3,1601,1,56.4958,5,11.299160
6,2,S.O.C. 14879,0,73.5000,5,14.700000
7,3,382652,0,29.1250,5,5.825000
8,1,113760,1,120.0000,4,30.000000
9,3,349909,0,21.0750,4,5.268750


In [ ]:
# adesso dovrei portare i valori della nuova colonna Sing_Fare del dataset df_pcl_tick_fare_count nel dataframe df(naturalmente creando una nuova feature chiamata sempre Sing_Fare) --> per portarceli/associarceli creeremo ed utilizzeremo un dizionario(result_dict) di appoggio con chiave il numero del Ticket e con valore il valore della nuova colonna Sing_Fare

In [8]:
# result_dict sara il dizionario che conterrà le coppie di valori date dai valori di Ticket e Sing_Fare
result_dict={}
# converto le features Ticket e Sing_Fare(sono le features con cui avro a che fare) in liste --> questa operazione è consigliata visto che le liste sono + semplici da monipolare
ticket_list= df_pcl_tick_fare_count['Ticket'].to_list()
sing_fare= df_pcl_tick_fare_count['Sing_Fare'].to_list()
# scorro la lunghezza del datframe, qui ho messo la lunghezza della feature Ticket ma avrei potuto mettere anche solo la lunghezza dell'intero dataframe visto che il numero di valori è lo stesso 
for i in range(0,len(df_pcl_tick_fare_count['Ticket'])):
    result_dict[ticket_list[i]]=sing_fare[i]
# print(result_dict)

# come mostrato sotto(se decommentiamo print(result_dict)) abbiamo popolato il dizionario result_dict con i valori sopra citati

In [9]:
# N.B. --> leggi cella sotto prima di comprendere questo pezzo di codice
# tale funzione associa in corrispondenza i valori della feature Ticket(presenti in entrambi i dataset, cioè result_dict e df) con i valori della feature Sing_fare
def conv_fare(ticket_name):
    # se il valore in ingresso(ticket_name) è nelle chiavi(.keys()) del dizionario result_dict esegui il 1° return --> gli passo result_dict.keys() cosi in questo modo scorre tutte le chiavi del dizionario result_dict(visto che ci potrebbero essere piu chiavi uguali) ed appena la trovi restituisci il valore corrispettivo alla chiave
    if ticket_name in list(result_dict.keys()):
        # la linea sotto restituisce "il valore" del dizionario result_dict con chiave ticket_name, si accede alla chiave così da restituire il valore corrispettivo che nel nostro caso è dato dal prezzo del ticket
        return result_dict[ticket_name]
    else:
        return 0

In [10]:
# aggiungo i valori del dizionario result_dict nel dataset df originale(avvalendomi della funzione conv_fare sopra indicata) creando una nua nuova feature Sing_Fare nel dataset df --> uso la feature['Ticket'] poichè i valori di tale features sono quelli in comune con il dataset df ed il dizionario result_dict(che abbiamo utilizzato come appoggio per salvare i valori di sing_fare) 
df['Sing_Fare']= df['Ticket'].apply(lambda x: conv_fare(x))

# adesso abbiamo anche la nuova feature Sing_Fare nel dataframe df con i valori reali, quindi senza prezzi(Fare) di gruppo

In [32]:
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_conv,Sing_Fare
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,7.2500
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,71.2833


adesso posso calcolare la media dei dei prezzi dei biglietti per singola classe e capire se il prezzo ha determinato la scelta di una classe più tosto che un'altra

In [11]:
# media prezzi classe 1

# rispetto al codice sotto, che calcola la media sia sulla feature Pclass che sulla feature Sing_Fare, avrei potuto scrivere il seguente codice
# df.loc[df['Pclass']==1, 'Sing_Fare'].mean()

mean_fare_pclass_1= df.loc[(df['Pclass']==1), ['Pclass','Sing_Fare']]
mean_fare_pclass_1.mean()

Pclass        1.000000
Sing_Fare    59.950153
dtype: float64

In [44]:
# media prezzi classe 2

mean_fare_pclass_2= df.loc[(df['Pclass']==2), ['Pclass','Sing_Fare']]
mean_fare_pclass_2.mean()

Pclass        2.000000
Sing_Fare    17.302481
dtype: float64

In [45]:
# media prezzi classe 3

mean_fare_pclass_3= df.loc[(df['Pclass']==3), ['Pclass','Sing_Fare']]
mean_fare_pclass_3.mean()

Pclass       3.000000
Sing_Fare    9.122043
dtype: float64

In [46]:
df.loc[df['Pclass']==3, 'Sing_Fare'].mean()

9.122042710472297

guardando i valori possiamo dire che i prezzi hanno influenzato l'acquisto del biglietto visto che in:
- 1° classe la media dei prezzi è 61.37(circa)
- 2° classe la media dei prezzi è 17.30(circa)
- 3° classe la media dei prezzi è 9.12(circa)

riepilogando possiamo dire che la media dei prezzi(feature Fare) tra le differenti classi potrebbe essere una discriminante visto che il prezzo elevato del biglietto in 1° classe ha fatto si che i meno abbienti acquistassero i biglietti 

-------------------------------------------

sotto codice di esempio, sono dei frammenti che mi potranno essere utili in futuro

In [ ]:
df[(df['Age']>16) & (df['Age']<50)]

In [24]:
# media 1° classe con i diversi ticket esclusi, qui ne ho fatti 2, solo che non posso farlo per tutto....ci vorrebbe un metodo che li passa tutti --> cmq anche così non mi funziona xkè mi da lo stesso risultato di sopra
df.loc[(df['Pclass']==3) & ((df['Ticket'] != '347082') & (df['Ticket'] != '347088')),'Fare'].mean()

13.421012603305792

-----------------------------------------

In [33]:
# vediamo il nuovo dataframe in modo da controllare tutte le features e vedere come utilizzarle, anche perchè c'è Sing_Fare che è nuova
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_conv,Sing_Fare
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,7.2500
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,71.2833


In [12]:
# ho lasciato la features Cabin(contiene anche valori NaN), non so se toglierla????
X= df.drop(['id','PassengerId','Survived','Name','Ticket','Fare','Cabin','Embarked'], axis=1).values
y= df['Survived'].values

In [54]:
X.head(2)

,Pclass,Sex,Age,SibSp,Parch,Sing_Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833


sotto il dataset X, dopo il drop sopra sul dataset df, non sarebbe nemmeno un dataset poichè diventa un'array numpy, lo mostro solo per renderci conto

[Pclass] [Age] [SibSp] [Parch] [Sex_conv] [Sing_Fare]

In [13]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=0, test_size=.3, stratify=y)

cosa voglio fare?

- Pclass --> variabile categorica numerica, la lascio così com'è --> penso si debba fare il ohe, vediamo se lo faccio dopo
- Sex --> variabile categorica, devo portarla a numerica * --> l'ho fatto sopra
- Age --> rimpiazzare i valori mancanti
      --> scaling
- SibSp --> variabile categorica numerica, la lascio così com'è
- Parch --> variabile categorica numerica, la lascio così com'è
- Fare --> scaling

In [14]:
df['Parch'].value_counts()

0    663
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

[Pclass] --> variabile categorica, 3 classi(1,2,3) --> nessuna codifica visto che la 1° classe, a nostro avviso, risulta + sicura quindi lasciandola così ci sarà un sorta di ordinalità --> oppure per devo applicare il ohe per fargli prendere la cardinalità?
[Age] --> variabile numerica continua, nessun valore nullo, scaling
[SibSp] --> variabile discreta, scaling?
[Parch] --> variabile discreta, scaling?
[Sex] - [Sex_conv] --> variabile categorica, faccio il ohe giusto per vedere ome gira
[Sing_Fare] --> variabile continua, scaling

In [14]:
skf= StratifiedKFold()
min_max_sca= MinMaxScaler()
ohe= OneHotEncoder(drop='first')
smp_imp= SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [24]:
# MODELLI
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# creo un dizionario con l'elenco dei classificatori
clf= {'log_reg':LogisticRegression(),
      'dec_tr': DecisionTreeClassifier(),
      'svc':SVC()
     }

In [28]:
# hyperpar_models= {'log_reg': [[1,2,3,4,5,6,8,11,14,18],['euclidean', 'manhattan', 'minkowski'],['uniform', 'distance']],
#                   'dec_tr': [['newton-cg', 'lbfgs', 'liblinear'], ['l2'], [1, 10, 20, 50]],
#                   'svc': [['linear','rbf'], [0.1, 1, 10, 100], [0.1, 1, 10, 100, 1000]]}
hyperpar_models= {'log_reg': [{'penalty': 'l1','C': 0.5}, {'penalty': 'l1','C': 0.1}],
                  'dec_tr': [{'criterion': 'gini','max_depth': 6}, {'criterion': 'gini','max_depth': 10}],
                  'svc': [{'C': 1, 'kernel': 'rbf'}, {'C': 0.5, 'kernel': 'poly'}]}


In [29]:
for key, v in clf.items():
    for par in hyperpar_models[key]:
        print(par)
        print(key)
    # v.set_params(**{'penalty': 'l1','C': 0.5})

{'penalty': 'l1', 'C': 0.5}
log_reg
{'penalty': 'l1', 'C': 0.1}
log_reg
{'criterion': 'gini', 'max_depth': 6}
dec_tr
{'criterion': 'gini', 'max_depth': 10}
dec_tr
{'C': 1, 'kernel': 'rbf'}
svc
{'C': 0.5, 'kernel': 'poly'}
svc


In [17]:
# col_df --> nomi colonne che utilizzo quando nella funzione trasformo l'array numpy in dataframe per applicarvi le varie modifiche
col_df= ['Pclass','Sex','Age','SibSp','Parch','Sing_Fare']
# col_ohe= df[['Pclass','Sex','SibSp','Parch','Embarked']].columns.to_list()
# visto che col_ohe restituisce una lista(dopo avergli passato ".columns.to_list()"), a col_scal gliela passo direttamente
col_scal= ['Age','Sing_Fare']

In [18]:
# funzione da richiamare
def trans_conv(X_train_train, X_train_val, col_df_train, col_age, col_df_ohe, col_df_scal, ohe, min_max_sca):
    # 1° step --> encoding sul train
    # trasformo, di nuovo, l'array numpy in dataframe(così da potergli applicare tutte le conversioni che voglio) passandogli tutti i nomi delle colonne
    X_train_df= pd.DataFrame(X_train_train, columns=col_df_train)
    # rimpiazziamo i valori NaN della feature Age, quindi gli passo solo la feature Age
    # 1° step di conversioni --> feature Age, applico un imputer poichè ci sono dei valori nan da riempire
    # sotto applico il reshape poichè si tratta di una singola colonna, se fossero state almeno 2 non sarebbe stato necessario, almeno credo
    X_train_df[col_age]= smp_imp.fit_transform(X_train_df[col_age].values.reshape(-1,1))
    # scaling sulle features Age e Sing_Fare
    X_train_df[col_df_scal]= min_max_sca.fit_transform(X_train_df[col_df_scal])
    # ohe su Sex(anche se non ha senso visto che i valori sono solo 2, potevo fare una conversione fuori dal modello, ho fatto il ohe per capire come bisognava farlo internamente), lo faccio associandolo ad un nuovo dataframe, anche in questo caso abbiamo messo il reshape visto che si tratava di una singola variaible, capire il todense???????
    cat_enc= pd.DataFrame(ohe.fit_transform(X_train_df[col_df_ohe].values.reshape(-1,1)).todense())
    # concateno il dataframe(del ohe) appena creato con quello principale
    X_train_df= pd.concat([X_train_df, cat_enc], axis=1)
    # droppiamo la vecchia colonna di Sex, visto ceh adesso abbiamo le nuove colonne ohe di Sex
    X_train_df.drop(col_df_ohe, inplace=True, axis=1)
    # riporto il dataframe ad array, come all'inizio, poichè il modello lavora sugli indici
    X_train_arr= X_train_df.values
    # 2° step --> encoding sul validation, stessa cosa di sopra ma applicato al dataset di validation
    X_val_df= pd.DataFrame(X_train_val, columns=col_df_train)
    X_val_df[col_age]= smp_imp.transform(X_val_df[col_age].values.reshape(-1,1))
    X_val_df[col_df_scal]= min_max_sca.transform(X_val_df[col_df_scal])
    cat_enc=pd.DataFrame(ohe.transform(X_val_df[col_df_ohe].values.reshape(-1,1)).todense())
    X_val_df= pd.concat([X_val_df, cat_enc], axis=1)
    X_val_df.drop(col_df_ohe, inplace=True, axis=1)
    X_val_arr= X_val_df.values
    return X_train_arr, X_val_arr

In [20]:
df.head(2)

,id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sing_Fare
0,1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,7.2500
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,71.2833


In [25]:
df.describe()

,id,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sing_Fare
count,876.0,876.000000,876.000000,876.000000,707.000000,876.000000,876.000000,876.000000,876.000000
mean,1.0,444.794521,0.389269,2.315068,29.645219,0.531963,0.388128,32.755650,23.369303
std,0.0,257.806393,0.487863,0.835663,14.558872,1.110009,0.811374,49.936826,32.365562
min,1.0,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,4.012500,3.711100
25%,1.0,220.750000,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000,7.795800
50%,1.0,444.500000,0.000000,3.000000,28.000000,0.000000,0.000000,14.500000,10.500000
75%,1.0,667.250000,1.000000,3.000000,38.000000,1.000000,0.000000,31.275000,26.550000
max,1.0,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,247.520800


In [ ]:
df['Age'].value_counts()

In [28]:
print(X_train[0])

[ 3.   45.    1.    4.    4.65]


In [19]:
# modello --> DECISION TREE
result_cv_train_dt= []
result_cv_val_dt= []
cv_train_dt_fin= []
cv_val_dt_fin= []
diff_train_eval= []
models_result= {'model':[],
                'train_value':[],
                'val_value':[],
                'diff_train_eval':[]
                }
for model in clf:
    for train, val in skf.split(X_train, y_train):
        # la riga di codice sotto la scrivo poichè prima di passare i valori all'addestramento(fit) li modifico(gli applico le trasformazioni) passandoli alla funzione trans_conv --> infatti gli passo solo la parte delle variaibili indipendenti(per capirci la X) visto che alla variabile target non devo applicare nessuna modifica, infatti poi il tutto(compreso la variabile y la passo alla riga successiva dove faccio il fit)
        X_train_arr, X_val_arr= trans_conv(X_train[train], X_train[val], col_df, 'Age', 'Sex', col_scal, ohe, min_max_sca)
        # fit sul train
        clf_mod= model.fit(X_train_arr, y_train[train])
        # predizione sul dt train
        X_train_predict_dt= clf_mod.predict(X_train_arr)
        X_train_dt_f1= f1_score(X_train_predict_dt, y_train[train])
        result_cv_train_dt.append(X_train_dt_f1)
        # predizione sul dt validation
        X_val_predict_dt= clf_mod.predict(X_val_arr)
        X_val_dt_f1= f1_score(X_val_predict_dt, y_train[val])
        result_cv_val_dt.append(X_val_dt_f1)
    models_result['model'].append(model)
    models_result['train_value'].append(np.asarray(result_cv_train_dt).mean())
    models_result['val_value'].append(np.asarray(result_cv_val_dt).mean())
    sub_train_val= map(lambda x,y: x-y,models_result['train_value'], models_result['val_value'])
    models_result['diff_train_eval']= sub_train_val
res_fin= pd.DataFrame(models_result)

START NEW CODE

In [ ]:
def hyperpar_tun():

In [41]:
# modello --> DECISION TREE
result_cv_train_dt= []
result_cv_val_dt= []
cv_train_dt_fin= []
cv_val_dt_fin= []
models_result= {'model':[],
                'train_value':[],
                'val_value':[],
                'diff_train_eval':[]
                }
# selezioniao il modello
for model in clf:
    # in base al modello selezionato selezioniamo gli iperparametri
    if model == 'log_reg':
        for i in itertools.product(*hyperpar_lr):
    elif model == 'dec_tr':
        for j in itertools.product(*hyperpar_dt):
    else:
        for k in itertools.product(*hyperpar_svc):     

    for train, val in skf.split(X_train, y_train):
        # la riga di codice sotto la scrivo poichè prima di passare i valori all'addestramento(fit) li modifico(gli applico le trasformazioni) passandoli alla funzione trans_conv --> infatti gli passo solo la parte delle variaibili indipendenti(per capirci la X) visto che alla variabile target non devo applicare nessuna modifica, infatti poi il tutto(compreso la variabile y la passo alla riga successiva dove faccio il fit)
        X_train_arr, X_val_arr= trans_conv(X_train[train], X_train[val], col_df, 'Age', 'Sex', col_scal, ohe, min_max_sca)

        if model == 'log_reg':
            for i in itertools.product(*hyperpar_lr):


        clf_mod= model.fit(X_train_arr, y_train[train])
        # predizione sul dt train
        X_train_predict_dt= clf_mod.predict(X_train_arr)
        X_train_dt_f1= f1_score(X_train_predict_dt, y_train[train])
        result_cv_train_dt.append(X_train_dt_f1)
        # predizione sul dt validation
        X_val_predict_dt= clf_mod.predict(X_val_arr)
        X_val_dt_f1= f1_score(X_val_predict_dt, y_train[val])
        result_cv_val_dt.append(X_val_dt_f1)
    models_result['model'].append(model)
    models_result['train_value'].append(np.asarray(result_cv_train_dt).mean())
    models_result['val_value'].append(np.asarray(result_cv_val_dt).mean())
    sub_train_val= map(lambda x,y: x-y,models_result['train_value'], models_result['val_value'])
    models_result['diff_train_eval']= sub_train_val
res_fin= pd.DataFrame(models_result)

In [21]:
for x in itertools.product(*hyperpar_lr):
    print(x)

(1, 'euclidean', 'uniform')
(1, 'euclidean', 'distance')
(1, 'manhattan', 'uniform')
(1, 'manhattan', 'distance')
(1, 'minkowski', 'uniform')
(1, 'minkowski', 'distance')
(2, 'euclidean', 'uniform')
(2, 'euclidean', 'distance')
(2, 'manhattan', 'uniform')
(2, 'manhattan', 'distance')
(2, 'minkowski', 'uniform')
(2, 'minkowski', 'distance')
(3, 'euclidean', 'uniform')
(3, 'euclidean', 'distance')
(3, 'manhattan', 'uniform')
(3, 'manhattan', 'distance')
(3, 'minkowski', 'uniform')
(3, 'minkowski', 'distance')
(4, 'euclidean', 'uniform')
(4, 'euclidean', 'distance')
(4, 'manhattan', 'uniform')
(4, 'manhattan', 'distance')
(4, 'minkowski', 'uniform')
(4, 'minkowski', 'distance')
(5, 'euclidean', 'uniform')
(5, 'euclidean', 'distance')
(5, 'manhattan', 'uniform')
(5, 'manhattan', 'distance')
(5, 'minkowski', 'uniform')
(5, 'minkowski', 'distance')
(6, 'euclidean', 'uniform')
(6, 'euclidean', 'distance')
(6, 'manhattan', 'uniform')
(6, 'manhattan', 'distance')
(6, 'minkowski', 'uniform')
(6,

In [43]:
# ordiniamo per valore minore di diff_train_eval così restituisce subito il modello migliore
res_fin.sort_values(by=['diff_train_eval'])

,model,train_value,val_value,diff_train_eval
0,LogisticRegression(),0.741249,0.725011,0.016239
2,SVC(),0.825336,0.707940,0.117397
1,DecisionTreeClassifier(),0.862149,0.700368,0.161781


END NEW CODE